In [1]:
import pandas
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive
!pip install patool
import patoolib
import scipy.io
import glob
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
drive.mount('/content/gdrive')
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 

     |████████████████████████████████| 77 kB 4.0 MB/s 
Mounted at /content/gdrive


In [31]:
sim_dir =  '/content/gdrive/My Drive/פרויקט מסכם/new_simulations/'

for rar in glob.glob(sim_dir + '*.rar'):
  patoolib.extract_archive(rar, outdir=sim_dir)

patool: Extracting /content/gdrive/My Drive/פרויקט מסכם/new_simulations/two_ellipses.rar ...
patool: running /usr/bin/unrar x -- "/content/gdrive/My Drive/פרויקט מסכם/new_simulations/two_ellipses.rar"
patool:     with cwd='/content/gdrive/My Drive/פרויקט מסכם/new_simulations/'
patool: ... /content/gdrive/My Drive/פרויקט מסכם/new_simulations/two_ellipses.rar extracted to `/content/gdrive/My Drive/פרויקט מסכם/new_simulations/'.
patool: Extracting /content/gdrive/My Drive/פרויקט מסכם/new_simulations/two_both.rar ...
patool: running /usr/bin/unrar x -- "/content/gdrive/My Drive/פרויקט מסכם/new_simulations/two_both.rar"
patool:     with cwd='/content/gdrive/My Drive/פרויקט מסכם/new_simulations/'
patool: ... /content/gdrive/My Drive/פרויקט מסכם/new_simulations/two_both.rar extracted to `/content/gdrive/My Drive/פרויקט מסכם/new_simulations/'.
patool: Extracting /content/gdrive/My Drive/פרויקט מסכם/new_simulations/two_spheres.rar ...
patool: running /usr/bin/unrar x -- "/content/gdrive/My Driv

In [32]:
dataset = []
labels = []
i = 0
for file in glob.glob(sim_dir + '*/*.mat'):
  i += 1
  if i%100==0:
    print(i)
  mat = scipy.io.loadmat(file)['logY'].toarray()
  dataset.append(mat)
  if "homogenous" in file:
    labels.append(0)
  else: 
    labels.append(1)
dataset = np.real(np.array(dataset))
labels = np.array(labels)
# np.save(sim_dir + "dataset.npy", dataset)
# np.save(sim_dir + "labels.npy", labels)

# dataset = np.load('/content/gdrive/My Drive/פרויקט מסכם/simulations/dataset.npy')
# labels = np.load('/content/gdrive/My Drive/פרויקט מסכם/simulations/labels.npy')

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900


In [40]:
print(dataset.shape)
print(labels.shape)

for i in range(1,5):
  print(x_train_norm[i])
  print(t_train[i])

(3950, 25, 25)
(3950,)
[[-60.85622568 -60.99057231 -63.65863061 -67.13375523 -72.54900723
  -62.74010584 -63.55420307 -70.29202557 -70.68886576 -72.10316899
  -68.38899008 -68.3868372  -73.19105458 -72.60074619 -74.58317259
  -66.47490135 -69.37605091 -74.41269664 -77.93049431 -79.9633968
  -71.02314118 -74.19981128 -75.82366787 -76.42529571 -80.14572535]
 [-63.97063217 -62.36878346 -64.16000938 -66.81320866 -68.72249533
  -65.15241942 -66.25489306 -68.91403471 -69.72702077 -70.57841291
  -66.19972936 -70.07228343 -72.29697895 -73.00888598 -72.82241757
  -69.34308307 -72.62202708 -75.72079358 -75.07956696 -74.71858416
  -72.60286081 -75.3959111  -74.95829501 -75.61841635 -78.09347013]
 [-70.87091866 -67.14281186 -68.38352382 -67.60153699 -70.83200269
  -70.58945768 -70.60417877 -71.96234487 -69.33664814 -69.75086788
  -72.26661462 -74.33769668 -72.4389634  -70.8040552  -71.21113391
  -75.70471426 -76.10343998 -75.66497642 -73.9821889  -71.59706468
  -77.39574821 -75.56432763 -75.461880

In [34]:
def norm_data(data):
  output = []
  for sample in data: 
    sample = (sample - np.mean(sample))/np.std(sample)
    output.append(np.array(sample))
  return np.array(output)

input_size = 25*25
num_of_samples = dataset.shape[0]

perm = torch.randperm(num_of_samples)
dataset = dataset[perm]
labels = labels[perm]

x_train, x_val, x_test = dataset[:round(0.7*num_of_samples)], dataset[round(0.7*num_of_samples):round(0.9*num_of_samples)], dataset[round(0.9*num_of_samples):]
t_train, t_val, t_test = labels[:round(0.7*num_of_samples)],  labels[round(0.7*num_of_samples):round(0.9*num_of_samples)],  labels[round(0.9*num_of_samples):]

x_train_norm = norm_data(x_train)
x_val_norm = norm_data(x_val)
x_test_norm = norm_data(x_test)

x_train_norm = np.reshape(x_train_norm, (x_train_norm.shape[0], x_train_norm.shape[1]*x_train_norm.shape[2]))
x_val_norm = np.reshape(x_val_norm, (x_val_norm.shape[0], x_val_norm.shape[1]*x_val_norm.shape[2]))
x_test_norm = np.reshape(x_test_norm, (x_test_norm.shape[0], x_test_norm.shape[1]*x_test_norm.shape[2]))

train_loader = torch.utils.data.DataLoader(
    np.concatenate((t_train[:,None], x_train_norm),axis=1),
    batch_size=64, shuffle=True)

val_loader = torch.utils.data.DataLoader(
    np.concatenate((t_val[:,None], x_val_norm),axis=1),
    batch_size=64, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    np.concatenate((t_test[:,None], x_test_norm),axis=1),
    batch_size=64, shuffle=True)

In [56]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size=9, num_hidden=50):
        super(NeuralNetwork, self).__init__()
        self.layer1 = nn.Linear(input_size, num_hidden)
        self.layer2 = nn.Linear(num_hidden, num_hidden//2)
        self.layer3 = nn.Linear(num_hidden//2, 2)
        self.num_hidden = num_hidden
        self.input_size = input_size
    def forward(self, x):
        x = x.reshape([-1, input_size])
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = F.log_softmax(x, dim=1)
        return x 

class CNN(nn.Module):
    def __init__(self, input_size, n_feature):
        super(CNN, self).__init__()
        self.n_feature = n_feature
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=n_feature, kernel_size=5, padding='same')
        self.conv2 = nn.Conv2d(n_feature, n_feature, kernel_size=5, padding='same')
        self.fc1 = nn.Linear(n_feature*6*6, 50)
        self.fc2 = nn.Linear(50, 2)
        
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=2)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=2)
        x = x.view(-1, self.n_feature*6*6)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.log_softmax(x, dim=1)
        return x

In [44]:
def get_accuracy(model, loader=train_loader):
    model.eval()
    loss = 0
    correct = 0
    pred_list = []
    true_list = []
    for ar in loader:
        data = ar[:,1:]
        label = ar[:,0]
        # send to device
        data, label = data.to(device), label.to(device)

        data = data.view(-1, 25*25)
        data = data.view(-1, 1, 25, 25)
        
        pred = model(data)
        loss += F.nll_loss(pred, label.long(), reduction='sum').item() # sum up batch loss                                                               
        pred = pred.data.max(1, keepdim=True)[1] # get the index of the max log-probability                                                                 
        correct += pred.eq(label.data.view_as(pred)).cpu().sum().item()
    # print(f"correct = {correct}")
    # print(f"num_of_ars = {num_of_ars}")
    loss /= len(loader.dataset)
    accuracy = 100. * correct / len(loader.dataset)
    
    return loss, accuracy

def train(model, lr=0.01, max_iters=1000,num_epochs=6):
    model.train()
    train_accs, valid_accs = [], []
    epochs = []
    optimizer = optim.Adam(model.parameters(), lr=lr)
    n = 0 # the number of iterations
    iters, losses = [], []
    iters_sub = []

    for epoch in range(0, num_epochs):

        for batch_idx, ar in enumerate(train_loader):           
            data = ar[:,1:]
            label = ar[:,0]
            # send to device
            data, label = data.to(device), label.to(device)
            data = data.view(-1, 25*25)
            data = data.view(-1, 1, 25, 25)

            optimizer.zero_grad()
            pred = model(data)
            loss = F.nll_loss(pred, label.long())
            loss.backward()
            optimizer.step()

            iters.append(n)
            losses.append(loss)

            if batch_idx % 64 == 0: 
                
                iters_sub.append(n)
                train_loss, train_acc = get_accuracy(model, loader=train_loader)
                train_accs.append(train_acc)

                valid_loss, valid_acc = get_accuracy(model, loader=val_loader)
                valid_accs.append(valid_acc)
      
                print("Iter %d. [Val Acc %.0f%%] [Train Acc %.0f%%, Loss %f]" % (n, valid_acc, train_acc, train_loss))

            # increment the iteration number
            n += 1
            if n > max_iters:
                return iters, losses, iters_sub, train_accs, valid_accs
    return iters, losses, iters_sub, train_accs, valid_accs

In [57]:

model = CNN(input_size, 10)
model.to(device)

lr = 0.0001

iters, losses, iters_sub, train_accs, valid_accs = train(model.double(), lr=lr, max_iters=10000, num_epochs=15)

Iter 0. [Val Acc 88%] [Train Acc 88%, Loss 0.579108]
Iter 44. [Val Acc 88%] [Train Acc 88%, Loss 0.359694]
Iter 88. [Val Acc 88%] [Train Acc 88%, Loss 0.359808]
Iter 132. [Val Acc 88%] [Train Acc 88%, Loss 0.359652]
Iter 176. [Val Acc 88%] [Train Acc 88%, Loss 0.360552]
Iter 220. [Val Acc 88%] [Train Acc 88%, Loss 0.359725]
Iter 264. [Val Acc 88%] [Train Acc 88%, Loss 0.359262]
Iter 308. [Val Acc 88%] [Train Acc 88%, Loss 0.361069]
Iter 352. [Val Acc 88%] [Train Acc 88%, Loss 0.360424]
Iter 396. [Val Acc 88%] [Train Acc 88%, Loss 0.358970]
Iter 440. [Val Acc 88%] [Train Acc 88%, Loss 0.359096]
Iter 484. [Val Acc 88%] [Train Acc 88%, Loss 0.358871]
Iter 528. [Val Acc 88%] [Train Acc 88%, Loss 0.358816]
Iter 572. [Val Acc 88%] [Train Acc 88%, Loss 0.359579]
Iter 616. [Val Acc 88%] [Train Acc 88%, Loss 0.358590]
